# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
%tensorflow_version  2.x
import tensorflow as tf
import numpy as np
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

In [0]:
#Load the dataset:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [171]:
print ('X_train:',X_train.shape)
print ('y_train:',y_train.shape)

X_train: (50000, 32, 32, 3)
y_train: (50000, 1)


In [0]:
# Function for creating two data sets having classes from 0 to 4 and other having classes from 5 to 9
def prepare_dataset_class0to4(X_train, y_train, X_test, y_test):
    idx = np.where(y_train[:]<=4)[0]
    X_train_new = X_train[idx,::]
    y_train_new = y_train[idx,::]

    idx = np.where(y_test[:]<=4)[0]
    X_test_new = X_test[idx,::]
    y_test_new = y_test[idx,::]
    return X_train_new, y_train_new, X_test_new, y_test_new
    
def prepare_dataset_class5to9(X_train, y_train, X_test, y_test):
    idx = np.where(y_train[:]>=5)[0]
    X_train_new = X_train[idx,::]
    y_train_new = y_train[idx,::]

    idx = np.where(y_test[:]>=5)[0]
    X_test_new = X_test[idx,::]
    y_test_new = y_test[idx,::]
    return X_train_new, y_train_new, X_test_new, y_test_new

In [0]:
X_train_new1, y_train_new1, X_test_new1, y_test_new1 = prepare_dataset_class0to4(X_train, y_train, X_test, y_test)
X_train_new2, y_train_new2, X_test_new2, y_test_new2 = prepare_dataset_class5to9(X_train, y_train, X_test, y_test)

In [174]:
print ('X_train_new1:',X_train_new1.shape)
print ('X_train_new2:',X_train_new2.shape)
print ('y_train_new1:',y_train_new1.shape)
print ('y_train_new2:',y_train_new2.shape)
print ('X_test_new1:',X_test_new1.shape)
print ('X_test_new2:',X_test_new2.shape)
print ('y_test_new1:',y_test_new1.shape)
print ('y_test_new2:',y_test_new2.shape)

X_train_new1: (25000, 32, 32, 3)
X_train_new2: (25000, 32, 32, 3)
y_train_new1: (25000, 1)
y_train_new2: (25000, 1)
X_test_new1: (5000, 32, 32, 3)
X_test_new2: (5000, 32, 32, 3)
y_test_new1: (5000, 1)
y_test_new2: (5000, 1)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
from keras.utils import np_utils
num_classes = 10

y_train_new1 = np_utils.to_categorical(y_train_new1, num_classes)
y_train_new2 = np_utils.to_categorical(y_train_new2, num_classes)

y_test_new1 = np_utils.to_categorical(y_test_new1, num_classes)
y_test_new2 = np_utils.to_categorical(y_test_new2, num_classes)

In [176]:
print (y_train_new1.shape)
print (y_train_new2.shape)
print (y_test_new1.shape)
print (y_test_new2.shape)

(25000, 10)
(25000, 10)
(5000, 10)
(5000, 10)


In [177]:
indices,data = np.unique(y_train_new1, return_index=True)
print (indices)
print (data)

[0. 1.]
[0 4]


In [178]:
indices,data = np.unique(y_train_new2, return_index=True)
print (indices)
print (data)

[0. 1.]
[0 6]


In [179]:
indices,data = np.unique(y_test_new1, return_index=True)
print (indices)
print (data)

[0. 1.]
[0 3]


In [180]:

indices,data = np.unique(y_test_new2, return_index=True)
print (indices)
print (data)

[0. 1.]
[0 8]


### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [181]:
#Importing the necessary libraries 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, GlobalAveragePooling2D

#Building up a Sequential model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',input_shape = X_train_new1.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
global_average_pooling2d_6 ( (None, 64)               

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='scratchmodel.best.hdf5', 
                               verbose=1,save_best_only=True)

In [184]:
#Fitting the model on the train data and labels.
model.fit(X_train_new1, y_train_new1, batch_size=32, epochs=10, 
          verbose=1, callbacks=[checkpointer], validation_split=0.2, shuffle=True)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
19968/20000 [============================>.] - ETA: 0s - loss: 2.4521 - acc: 0.8398
Epoch 00001: val_loss improved from inf to 2.48216, saving model to scratchmodel.best.hdf5
20000/20000 [==============================] - 23s 1ms/sample - loss: 2.4519 - acc: 0.8398 - val_loss: 2.4822 - val_acc: 0.8382
Epoch 2/10
19936/20000 [============================>.] - ETA: 0s - loss: 2.4547 - acc: 0.8404
Epoch 00002: val_loss did not improve from 2.48216
20000/20000 [==============================] - 22s 1ms/sample - loss: 2.4536 - acc: 0.8405 - val_loss: 2.4822 - val_acc: 0.8382
Epoch 3/10
19968/20000 [============================>.] - ETA: 0s - loss: 2.4534 - acc: 0.8405
Epoch 00003: val_loss did not improve from 2.48216
20000/20000 [==============================] - 22s 1ms/sample - loss: 2.4536 - acc: 0.8405 - val_loss: 2.4822 - val_acc: 0.8382
Epoch 4/10
19936/20000 [============================>.] - ETA: 0s - loss: 2.4537 - acc: 0

In [185]:
#Evaluate the model on the test data
score = model.evaluate(X_test_new1, y_test_new1)

#Accuracy on test data
print('Accuracy on the Test Images: ', score[1])

5000/5000 [==============================] - 2s 351us/sample - loss: 2.4539 - acc: 0.8400
Accuracy on the Test Images:  0.8400001


In [0]:

initial_weights = model.get_weights()

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
transf_model = model

In [0]:
transf_model.set_weights(initial_weights)

In [189]:
transf_model.layers

In [190]:
transf_model.layers[7]

In [191]:
#Set all layers trainable to False (except dense layer)
for layer in transf_model.layers:
    layer.trainable = False
transf_model.layers[7].trainable = True
print(transf_model.layers[7])

In [0]:
#Compile model
transf_model.compile(loss="binary_crossentropy", optimizer='adam',\
    metrics=["accuracy"])

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [195]:
transf_model.fit(X_test_new2, y_test_new2, batch_size=32, epochs=10,
          validation_split=0.2, callbacks=[checkpointer], verbose=1, shuffle=True)

Train on 4000 samples, validate on 1000 samples
Epoch 1/10
3840/4000 [===========================>..] - ETA: 0s - loss: 0.3250 - acc: 0.8828
Epoch 00001: val_loss improved from 2.24055 to 0.31933, saving model to scratchmodel.best.hdf5
4000/4000 [==============================] - 2s 613us/sample - loss: 0.3253 - acc: 0.8824 - val_loss: 0.3193 - val_acc: 0.8828
Epoch 2/10
3968/4000 [============================>.] - ETA: 0s - loss: 0.3103 - acc: 0.8849
Epoch 00002: val_loss improved from 0.31933 to 0.31676, saving model to scratchmodel.best.hdf5
4000/4000 [==============================] - 2s 451us/sample - loss: 0.3112 - acc: 0.8846 - val_loss: 0.3168 - val_acc: 0.8863
Epoch 3/10
3968/4000 [============================>.] - ETA: 0s - loss: 0.3078 - acc: 0.8862
Epoch 00003: val_loss did not improve from 0.31676
4000/4000 [==============================] - 2s 463us/sample - loss: 0.3076 - acc: 0.8861 - val_loss: 0.3311 - val_acc: 0.8793
Epoch 4/10
3936/4000 [============================>

In [196]:
#Evaluate the model on the test data
score  = transf_model.evaluate(X_test_new2, y_test_new2)

#Accuracy on test data
print('Accuracy on the Test Images: ', score[1])

5000/5000 [==============================] - 2s 367us/sample - loss: 0.2748 - acc: 0.8908
Accuracy on the Test Images:  0.89081985


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import re
import numpy as np

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)


### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [6]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [7]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [8]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = TfidfVectorizer()
X_train = pd.DataFrame(vect.fit_transform(twenty_train.data[:]).toarray(), columns=vect.get_feature_names())

In [0]:
y_train = twenty_train.target

In [0]:
X_test = pd.DataFrame(vect.transform(twenty_test.data[:]).toarray(), columns=vect.get_feature_names())

In [0]:
y_test = twenty_test.target

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2257, 35788)
(1502, 35788)
(2257,)
(1502,)


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_class = logreg.predict(X_test)
print("Training Score: " , logreg.score(X_train, y_train))
print("Testing Score: " , logreg.score(X_test, y_test))

Training Score:  0.9920248116969429
Testing Score:  0.8974700399467377
